# TextRank Algorithm from Scratch

TextRank is a popular graph-based algorithm used for automatic text summarization and keyword extraction in Natural Language Processing (NLP). It is inspired by the PageRank algorithm used by Google to rank web pages in search results. TextRank represents a document as a graph, where each sentence (or word) is a node, and edges between nodes represent the relationship between sentences (or words). The algorithm assigns scores to each node, representing the importance or centrality of the sentence (or word) in the text.

This exercise takes you through the process of the TextRank algorithm. Here, we will
fetch the dataset and summarize the articles present in it with the help of the TextRank
algorithm.

In [2]:
# %pip install --upgrade numpy 
import numpy as np
import pandas as pd

import nltk
import re
# %pip install contractions
import contractions

The code also downloads the nltk Punkt tokenizer models. These will be used to
tokenize the sentences and words in the articles.

In [3]:
# nltk.download('punkt')
pd.set_option('display.max_colwidth', 1000)

In this step, we will load GloVe vector word representations, which are located in a
ZIP file. The code given here is used to extract the file:

In [4]:
import zipfile

GLOVE_DIR = 'datasets/glove/'
GLOVE_ZIP = GLOVE_DIR + 'glove.6B.50d.zip'

In [5]:
zip_ref = zipfile.ZipFile(GLOVE_ZIP, 'r')
zip_ref.extractall(GLOVE_DIR)
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='datasets/glove/glove.6B.50d.zip' mode='r'>>

After extracting the glove vectors, we need to load them into a dictionary. With
this dictionary, we can find the vector for each word.

In [6]:
def load_golve_vectors(fn):
    print("Loading GLove Model")
    with open(fn, 'r', encoding='utf-8') as glove_vector_file:
        model = {}
        for line in glove_vector_file:
            parts = line.split()
            word = parts[0]
            embeddings = np.array([float(val) for val in parts[1:]])
            model[word] = embeddings
        print("loaded {} words".format(len(model)))
    return model

In [7]:
glove_vectors = load_golve_vectors('datasets/glove/glove.6B.50d.txt')

Loading GLove Model
loaded 400000 words


Now we can read the tennis articles, which are located in a CSV file in the data
directory.

In [8]:
articles = pd.read_csv('datasets/tennis_articles_v4.csv')
articles.head(2)

,article_id,article_text,source
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours w...",http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/


In [24]:
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

the following code will download and load the NLTK English language stopwords:

The code given here creates a number of functions that will clean the article text,
as well as tokenizing it into words and sentences:

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize

CLEAN_PATTER = r'[^a-zA-Z\s]'

def clean(word):
    return re.sub(CLEAN_PATTER, ' ', word)

def clean_sentence(sentence):
    sentence = [clean(word) for word in sentence]
    return [word for word in sentence if word]

def clean_sentences(sentences):
    return [clean_sentence(sentence) for sentence in sentences]

def lower(sentence):
    return [word.lower() for word in sentence]

def remove_stopwords(sentence):
    words = [word for word in sentence if word not in stop_words]
    return [word for word in words if len(word) > 0]

def tokenize_words(sentences):
    return [word_tokenize(sentence) for sentence in sentences]

def fix_contractions(sentences):
    return [contractions.fix(sentence) for sentence in sentences]

Now apply the functions we just created to the articles. This will split each article
into sentences and each sentence into words. Note that, because we are using
DataFrames, we can apply the functions to each of the seven articles in the
dataset:

In [11]:
articles['SentencesInArticle'] = articles.article_text.apply(sent_tokenize)

articles['WordsInSentences'] = articles.SentencesInArticle\
                .apply(fix_contractions)\
                .apply(lower)\
                .apply(tokenize_words)\
                .apply(remove_stopwords)\
                .apply(clean_sentences)

Now subset the article columns to see the SentencesInArticles and
WordInSentences columns. We will use the head() function to see the first two
records:

In [12]:
articles = articles[['SentencesInArticle', 'WordsInSentences']]
articles.head(2)

,SentencesInArticle,WordsInSentences
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour, ], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, , , i, do, not, really, hide, any, feelings, too, much, ], [i, think, everyone, knows, this, is, my, job, here, ], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, , i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, net so, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match, ], [i, am, a, pretty, competitive, girl, ], [i, say, my, hellos, , but, i, am, not, sending, any, players, flowers, as, well, ], [uhm, , i, am, not, really, friendly, or, close, to, many, players, ], [i, have, not, a, lot, of, friends, away, from, the, courts, , ], [wh..."
1,"[BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday., Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday., Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1., He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand., He clinched on his fourth chance when Medvedev netted from the baseline., Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal., The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent., Copil has two after also beating No., 6 Marin Cilic in the second round., Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1...","[[basel, , switzerland, , ap, , , roger, federer, advanced, to, the, th, swiss, indoors, final, of, his, career, by, beating, seventh seeded, daniil, medvedev, , , , on, saturday, ], [seeking, a, ninth, title, at, his, hometown, event, , and, a, th, overall, , federer, will, play, th ranked, marius, copil, on, sunday, ], [federer, dominated, the, th ranked, medvedev, and, had, his, first, match point, chance, to, break, serve, again, at, , ], [he, then, dropped, his, serve, to, love, , and, let, another, match, point, slip, in, medvedev, s, next, service, game, by, netting, a, backhand, ], [he, clinched, on, his, fourth, chance, when, medvedev, netted, from, the, baseline, ], [copil, upset, expectations, of, a, federer, final, against, alexander, zverev, in, a, , , , , , , , , win, over, the, fifth ranked, german, in, the, earlier, semifinal, ], [the, romanian, aims, for, a, first, title, after, arriving, at, basel, without, a, career, ..."


he next step is to create the sentence vectors. We define the following functions,
namely sentence_vector() and sentences_to_vectors(). We will also use a vector
size of 50:

In [13]:
VECTOR_SIZE = 50
EMPTY_VECTOR = np.zeros(VECTOR_SIZE)

def sentence_vector(sentence):
    return sum([glove_vectors.get(word, EMPTY_VECTOR)
               for word in sentence])/len(sentence)

def sentence_to_vectors(sentences):
    return [sentence_vector(sentence) for sentence in sentences]

Now that we have the function to create sentence vectors, we can run it. It will
create another column in the DataFrame, called SentenceVectors:

In [14]:
articles['SentenceVector'] = \
            articles.WordsInSentences.apply(sentence_to_vectors)

The next step is to create a similarity matrix. The similarity matrix captures the
degree to which one sentence is similar to another for a given article. The function
is as follows:

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_matrix(sentence_vectors):
    sim_mat = np.zeros([len(sentence_vectors), len(sentence_vectors)])
    for i in range(len(sentence_vectors)):
        for j in range(len(sentence_vectors)):
            element_i = sentence_vectors[i].reshape(1, VECTOR_SIZE)
            element_j = sentence_vectors[j].reshape(1, VECTOR_SIZE)
            sim_mat[i][j] = cosine_similarity(element_i, element_j)[0,0]
    return sim_mat

Now we run the function to create the similarity matrices for each article:

In [16]:
articles['SimMatrix'] = articles['SimMatrix']=\
        articles.SentenceVector.apply(similarity_matrix)

The step after creating similarity matrices is to create a graph from the matrix.
We use a Python library called networkx to create the graph. The graph will help
us determine the relative importance of each sentence based on how much it is
similar to other sentences:

In [17]:
import networkx as nx

def compute_graph(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

In [18]:
# to create the new column with the graph for that article

articles['Graph'] = articles.SimMatrix.apply(compute_graph)

Take a look at the articles using articles.head(). You can see each of the columns
that we added, all the way to the Graph column:

In [19]:
articles.head(2)

,SentencesInArticle,WordsInSentences,SentenceVector,SimMatrix,Graph
0,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour, ], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, , , i, do, not, really, hide, any, feelings, too, much, ], [i, think, everyone, knows, this, is, my, job, here, ], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, , i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, net so, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match, ], [i, am, a, pretty, competitive, girl, ], [i, say, my, hellos, , but, i, am, not, sending, any, players, flowers, as, well, ], [uhm, , i, am, not, really, friendly, or, close, to, many, players, ], [i, have, not, a, lot, of, friends, away, from, the, courts, , ], [wh...","[[0.06649714285714288, 0.48873992857142873, -0.21009307142857142, 0.38912571428571424, -0.08841421428571429, -0.09431128571428572, -0.37479500000000004, 0.37201680000000004, -0.3327198735714286, -0.10679078571428571, 0.10778792857142858, -0.13581192857142857, -0.6358756785714287, 0.013304142857142851, 0.7273357500000001, 0.04358028571428572, -0.10138370714285717, 0.050696714285714296, -0.27315264285714286, -0.17919542857142856, -0.22918825714285718, 0.28847542857142855, 0.010318428571428574, 0.2803609285714285, 0.11280642857142856, -1.233634285714286, -0.2712247142857143, -0.07170157142857143, -0.3025570714285714, -0.34274057142857145, 2.533484285714286, 0.2289164285714286, -0.03808821428571428, 0.06688921428571427, 0.021217137857142854, -0.010322864285714297, 0.26075207142857143, 0.43792228571428576, -0.01944572142857142, -0.5688658571428572, -0.00652885714285713, -0.08326750000000001, 0.16795221428571425, 0.06045814285714286, 0.1047477142857143, -0.02568685714285714, -0.063918064...","[[1.0, 0.880420442283511, 0.8400741992486214, 0.9129596149615925, 0.8445816500505549, 0.8497893583785371, 0.8768521662830951, 0.8862589251673211, 0.8766881842306722, 0.9280216219887607, 0.8707611906386944, 0.8895783304964502, 0.8313178033584997, 0.8565959297474461, 0.8824241286286073, 0.8868385761498726, 0.8136190246763895], [0.880420442283511, 0.9999999999999999, 0.9408964676641774, 0.9686965549113967, 0.8942781393973298, 0.9586334393985559, 0.9585076101382632, 0.9730773874995188, 0.987312380978821, 0.9197064789163073, 0.9469263577646915, 0.9640488970318377, 0.9632944418681552, 0.9732690670320118, 0.9616582246780773, 0.9741510958493368, 0.9564473923061918], [0.8400741992486214, 0.9408964676641774, 1.0, 0.9303313796847563, 0.9448434579223796, 0.9570165239566268, 0.9414391114508267, 0.9183752804522204, 0.9549370028865451, 0.845894499773654, 0.8930468970782345, 0.9547127695525697, 0.9563604609626242, 0.9106663731073267, 0.969

The graph contains a score and a numeric index to a sentence. We have to write
a function that will rank the scores and return the top n sentences by their graph
scores:

In [20]:
def get_ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i], s)
                       for i, s in enumerate(sentences)), reverse=True)
    top_n_scores = [sentence for score, sentence in top_scores[:n]]
    return " ".join(top_n_scores)

ter creating the function, we can apply it to the DataFrame to create a new
column, called Summary. This will contain the top three sentences for each article:

In [21]:
articles['Summary'] = articles.apply(lambda d: get_ranked_sentences(d.SentencesInArticle,d.Graph), axis=1)

In [22]:
# Now you can see the summary for each article:
articles.loc[0].Summary

"I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players. But ultimately tennis is just a very small part of what we do. I think everyone just thinks because we're tennis players we should be the greatest of friends."

In [23]:
# To check the summary of ID 1, type the following code:
articles.loc[1].Summary

'Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday.'